In [1]:
"""
Name: example_analysis_data_creation.ipynb
Authors: Stephan Meighen-Berger
Example how to construct a frequency analysis
"""

'\nName: example_analysis.ipynb\nAuthors: Stephan Meighen-Berger\nExample how to construct a frequency analysis\n'

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import imageio
from tqdm import tqdm
import pickle

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day, config

In [6]:
# Some example settings
config['scenario']['population size'] = 100
config['scenario']['duration'] = 6000
config['scenario']['organism movement'] = False
config['scenario']['exclusion'] = True
config['scenario']['injection']['rate'] = 1
config['scenario']['injection']['y range'] = [0., 10.]
config['scenario']['light prop'] = {
            "switch": True,
            "x_pos": 5.,
            "y_pos": 10.,
        }
config['scenario']['detector'] = {
    "switch": True,
    "type": "PMTSpec",
    "response": True,
    "acceptance": "Flat",
    "mean detection prob": 0.5
}
# Organisms 
config['organisms']['emission fraction'] = 0.1
config['organisms']['alpha'] = 2.
config['organisms']['photon yield'] = 1e10
# Geometry
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    'offset': None,
}
config['geometry']['observation'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    "offset": np.array([0., 0.]),
}
config['geometry']["exclusion"] = {
    "function": "sphere",
    "radius": 0.15,
    "x_pos": 2.,
    "y_pos": 5.,
}
# Water
config['water']['model']['name'] = 'custom' # 'potential cylinder'
config['water']['model']['directory'] = "../data/current/long_run/"
config['water']['model']['time step'] = 0.1
config['advanced']['starting step'] = 0

In [7]:
# Which detectors we want to use
wavelengths = {
    "Detector 1": ["1", "#4575b4"],
    "Detector 5": ["2", "#91bfdb"],
    "Detector 8": ["3", "#e0f3f8"],
    "Detector 3": ["4", "#fee090"],
    "Detector 10": ["5", "#fc8d59"],
}

In [ ]:
# Launching multiple simulations to use in the analysis
seeds = np.arange(100)
counter = 0
for seed in tqdm(seeds):
    # General
    config["general"]["random state seed"] = seed
    # Creating a fourth_day object
    fd = Fourth_Day()
    # Launching solver
    fd.sim()
    # Fetching relevant data
    # Totals
    total = fd.measured["Detector 1"].values
    for detector in wavelengths.keys():
        if detector == "Detector 1":
            continue
        fd.measured[detector].values
        total += fd.measured[detector].values
    storage_dic = {
        "time": fd.t,
        "data": total
    }
    pickle.dump(storage_dic, open("../data/storage/run_%d.p" % counter, "wb"))
    counter += 1

 25%|██████████████████▌                                                       | 25/100 [7:44:01<24:18:05, 1166.47s/it]

In [ ]:
# Plotting standards
std_size = 3.
fontsize = 10.
lw=1.
h_length=0.2

In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
from itertools import groupby
def scramble_data(data, toprint=False):
    peaks = np.array([list(v) for k,v in groupby(data, key = lambda x: x != 0) if k != 0])
    number_of_peaks = len(peaks)
    number_of_elements = np.sum(np.array([len(it) for it in peaks]))
    total_number_of_elements = data.shape[0]
    number_of_pauses = total_number_of_elements - number_of_elements
    peak_order = np.arange(len(peaks))
    np.random.shuffle(peak_order)
    if toprint:
        print(number_of_pauses)
        print(number_of_peaks)
    peak_pos = np.random.choice(number_of_pauses, size=number_of_peaks, replace=False)
    peak_pos = np.sort(peak_pos)
    peak_diffs = np.diff(peak_pos)
    zero_arrs = np.array([
        np.zeros(peak_diff)
        for peak_diff in peak_diffs
    ])
    # Need to add 0
    to_add = np.zeros(number_of_pauses - np.sum(peak_diffs))
    list_form = zero_arrs.tolist()
    list_form.append(to_add)
    zero_arrs = np.array(list_form)
    splitting_base = np.array([
        np.concatenate((zero_arrs[i], peaks[i]))
        for i in peak_order
    ])
    new_data = np.array([item for sublist in splitting_base for item in sublist])
    return new_data

In [ ]:
# Loading data
counter = 0
totals = []
for seed in seeds:
    totals.append(pickle.load(open("../data/storage/run_%d.p" % counter, "rb"))["data"])
    counter += 1
totals = np.array(totals)

In [ ]:
# Scambling multiple times
scrambled_array = np.array([
    scramble_data(totals[i], toprint=False)
    for i in range(len(totals))
])
# The fft
fft_det_data = np.fft.fft(totals, norm='ortho', axis=1)
fft_freq_data = np.fft.fftfreq(totals[0].size, d=0.1)
fft_det_bkgrd = np.fft.fft(scrambled_array, norm='ortho', axis=1)
fft_freq_bkgrd = np.fft.fftfreq(scrambled_array[0].size, d=0.1)
# The mean and std
mean_arr_data = np.mean(np.abs(fft_det_data.real) / 1e7, axis=0, dtype=np.float64)
mean_arr_bkgrd = np.mean(np.abs(fft_det_bkgrd.real) / 1e7, axis=0, dtype=np.float64)
std_arr_bkgrd = np.std(np.abs(fft_det_bkgrd.real) / 1e7, axis=0, dtype=np.float64)

In [ ]:
# Energy distributions
figure, (ax1) = plt.subplots(1, 1, figsize=(std_size, std_size * 6. / 8.), sharex=True)
cut_off = np.argmax(fft_freq_data)
ax1.fill_between(fft_freq_bkgrd[:cut_off],
                 mean_arr_bkgrd.real[:cut_off] - 2. * std_arr_bkgrd[:cut_off] * mean_arr_bkgrd.real[:cut_off],
                 mean_arr_bkgrd.real[:cut_off] + 2. * std_arr_bkgrd[:cut_off] * mean_arr_bkgrd.real[:cut_off],
                 color='#009C3B', alpha=1.)
ax1.fill_between(fft_freq_bkgrd[:cut_off],
                 mean_arr_bkgrd.real[:cut_off] - std_arr_bkgrd[:cut_off] * mean_arr_bkgrd.real[:cut_off],
                 mean_arr_bkgrd.real[:cut_off] + std_arr_bkgrd[:cut_off] * mean_arr_bkgrd.real[:cut_off],
                 color='#FFDF00', alpha=1.)
ax1.plot(fft_freq_data[:cut_off],
         mean_arr_data.real[:cut_off],
         lw=lw, color='k')
ax1.plot(fft_freq_bkgrd[:cut_off],
         mean_arr_bkgrd.real[:cut_off],
         lw=lw, color='r')
ax1.set_xscale('linear')
ax1.set_yscale('linear')
ax1.set_xlabel(r'$\nu\;[\mathrm{Hz}]$', fontsize=fontsize)
ax1.set_ylabel(r'$|\mathrm{FFT}(N_\gamma)| / 1e7$', fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
# ax1.grid(True)
ax1.set_ylim(0.,1.)
ax1.set_xlim(0., 5.)
plt.tight_layout()
plt.show()

In [ ]:
figure.savefig(PICS + "MC_FFT.pdf",
               bbox_inches='tight')